# Monitoring Production Models with Amazon SageMaker Model Monitor and Clarify

#### Monitoring production machine learning (ML) models is a critical step to ensure that the models continue to meet business needs. Besides the infrastructure hosting the model, there are other important aspects of ML models that should be monitored regularly. As models age over a period of time, the real-world inference data distribution may change as compared to the data used for training the model. For example, consumer purchase patterns may change in the retail industry and economic conditions such as mortgage rates may change in the financial industry.

#### This gradual misalignment between the training and the live inference datasets can have a big impact on model predictions. Model quality metrics such as accuracy may degrade over time as well. Degraded model quality has a negative impact on business outcomes. Regulatory requirements, such as ensuring that ML models are unbiased and explainable, add another angle to model monitoring. Comprehensive monitoring of production models for these aspects allows you to proactively identify if and when a production model needs to be updated. Updating a production model needs both retraining and deployment resources. The costs involved in updating a production model should be weighed against the opportunity costs of effectively serving the model consumers.

#### Amazon SageMaker Model Monitor provides capabilities to monitor data drift and the model quality of models deployed as SageMaker real-time endpoints. Amazon SageMaker Clarify provides capabilities to monitor the deployed model for bias and feature attribution drift. Using a combination of these two features, you can monitor the following four different aspects of ML models deployed on SageMaker:

- Data drift: If the live inference traffic data served by the deployed model is statistically different from the training data the model was trained on, the model prediction accuracy will start to deteriorate. Using a combination of a training data baseline and periodic monitoring to compare the incoming inference requests with the baseline data, SageMaker Model Monitor detects data drift. Model Monitor further generates data drift metrics that are integrated with Amazon CloudWatch. Using these CloudWatch alerts, you can generate data drift detection alerts.

- Model quality: Monitoring model quality involves comparing labels predicted by a model to the actual labels, also called the ground truth inference labels. Model Monitor periodically merges data captured from real-time inferences with the ground truth labels to compare model quality drift against a baseline generated with training data. Similar to data drift metrics, model quality metrics are integrated with CloudWatch, so alerts can be generated if the model quality falls below a threshold.

- Bias drift: Statistically, significant drift between the live inference traffic data and the training data could also result in bias in the model over a period of time. This could happen even after detecting and addressing bias in the training data before training and deploying the model. SageMaker Clarify continuously monitors a deployed model for bias and generates bias metrics that are integrated with CloudWatch metrics.

- Feature attribution drift: Along with introducing bias in deployed models, drift in live inference data distribution can also cause drift in feature attribution values. Feature attribution ranks the individual features of a dataset according to their relative importance to a model trained using that dataset using an importance score. The feature importance score provides one way of explaining the model predictions by providing insight into which features played a role in making predictions. SageMaker Clarify compares the feature attribution or feature rankings in the training data to the feature attribution or feature rankings in live inference traffic data. Similar to other types of monitoring, feature attribution drift metrics are generated and integrated with CloudWatch.

#### 1. Enable data capture: The first step is to enable data capture on the real-time endpoint. On enabling data capture, input to and output from the SageMaker endpoint is captured and saved in Amazon Simple Storage Service (S3). Input captured includes the live inference traffic requests and output captured includes predictions from the deployed model. This is a common step for all four types of monitoring: data drift, model quality, bias drift, and feature attribution drift monitoring.

#### 2. Generate baseline: In this step, the training or validation data is analyzed to generate a baseline. The baseline generated will be further used in the next step to compare against the live inference traffic. The baseline generation process computes metrics about the data analyzed and suggests constraints for the metrics. The baseline generated is unique to the type of monitoring.

#### 3 Schedule and execute monitoring job: To continuously monitor the real-time endpoint, the next step is to create a monitoring schedule to execute at a predefined interval. Once the monitoring schedule is in place, SageMaker Processing jobs are automatically kicked off to analyze the data captured from the endpoint in a specific interval. For each execution of the monitoring job, the processing job compares live traffic data captured with the baseline. If the metrics generated on the live traffic data captured in a period are outside the range of constraints suggested by the baseline, a violation is generated. The scheduled monitoring jobs also generate monitoring reports for each execution, which are saved in an S3 bucket. Additionally, CloudWatch metrics are also generated, the exact metrics being unique to the type of monitoring.

#### Analyze and act on results: Reports generated by the monitoring job can either be downloaded directly from S3 or visualized in a SageMaker Studio environment. In the Studio environment, you can also visualize the details of the monitoring jobs and create charts that compare the baseline metrics with the metrics calculated by the monitoring job.

#### To remediate issues discovered, you can use the CloudWatch metrics emitted from the monitoring job. The specific metrics depend on the type of the monitoring job. You can configure CloudWatch alerts for these metrics, based on the threshold values suggested by the baseline job. CloudWatch alerts allow you to automate responses to violations and metrics generated by monitoring jobs.

## Data drift monitoring

#### You monitor a production model for data drift to ensure that the distribution of the live inference traffic the deployed model is serving does not drift away from the distribution of the dataset used for training the model.

#### Enable data capture for the deployed endpoint: The first step is to deploy a SageMaker endpoint with data capture enabled. As you can see from the following sample code, configuring data capture includes specifying the percentage of inference traffic to capture and the S3 location to save the captured traffic:

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig
data_capture_config = DataCaptureConfig(
enable_capture=True,
sampling_percentage=100,  destination_s3_uri=s3_capture_upload_path
)

#### To deploy the model, create the endpoint by passing in the data capture configuration as follows:

In [ ]:
predictor = model.deploy(initial_instance_count=1,
                instance_type='ml.m4.xlarge',
                endpoint_name=endpoint_name,
               data_capture_config = data_capture_config)

#### The following code shows a sample of the data captured. As you can see, both the request to and response from the endpoint along with event metadata are captured:

In [ ]:
{
"captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
     "data": "0,2020,12,4,31,0,19.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0\n",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "-4.902510643005371",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "e68592ca-948c-44dd-a764-608934e49534",

    "inferenceTime": "2021-06-28T18:41:16Z"
  },
  "eventVersion": "0"
}

#### Generate baseline: The second step is to configure and execute a data baseline job. This baseline job uses SageMaker Processing to analyze the training data at scale. For data drift monitoring, use DefaultModelMonitor to configure the infrastructure to execute the processing job on and the maximum runtime. Sample code is shown as follows:

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

#### Use the suggest_baseline method on DefaultModelMonitor to configure and kick off the baseline job. To configure the baseline job, specify where the baseline data is and where you want the baseline results to be saved in S3, as follows:

In [ ]:
my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri + "/training-dataset-with-header.csv",
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

#### The generated constraints also suggest completeness for each feature, which represents the percentage of values that can be non-null in the inference traffic. In this example, since completeness for all features is at 1.0, there cannot be any null values of these features in the inference traffic. Additionally, as suggested by num_constraints.is_non_negative, none of the integral and fractional features can be null.

#### Schedule and execute a data drift monitoring job: The third step is to configure and schedule a data drift monitoring job. To configure the data drift monitoring job, specify the endpoint to monitor, the location to store the monitoring results, the baseline statistics and constraints, and the schedule to execute the job on. The following sample code configures a monitoring job to be executed every hour:

In [ ]:
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True
)

#### The monitoring job emits CloudWatch metrics for all features included in the training data. Common metrics generated for all features are Completeness and BaselineDrift. The Completeness metric indicates the percentage of values that can be null for a given feature in a specific interval. The BaselineDrift metric indicates how much a feature has drifted in a specific interval from the baseline. Additionally, for numerical features, a few other metrics emitted are Max, Min, Sum, SampleCount, and AverageCount, as observed during the interval.

#### For any of these metrics, you can configure a CloudWatch alert to be triggered based on threshold values suggested in the constraints file. If the feature values in the inference traffic observed during a given interval violate the threshold values, an alert is raised.

#### Analyze and act on results: The final step is to analyze and act on the monitoring results. As mentioned in the high-level monitoring steps discussion earlier, you can download the monitoring reports from S3 and analyze them in your notebook environment or use Studio to view the monitoring details. For example, downloading the violation report to a notebook environment and viewing the report contents shows.

#### You can decide what actions you want to take on these alerts according to your business and operational requirements. You can automate actions such as updating the model, updating your training data, and retraining and updating the model as a response to the CloudWatch alert triggered.

## Model Quality Drift

#### Enable data capture for the deployed endpoint: The first step is to deploy a SageMaker endpoint with data capture enabled and capture predictions made by the model in an S3 bucket.

#### Generate baseline: The second step is baseline generation. While the baseline job for data drift directly analyzes the training dataset for data distribution statistics, the model quality baseline job compares the labels in a baseline dataset with the predictions made by the model. So, instead of using the training data directly, you have to first generate a baseline dataset consisting of labels by running predictions against the model. You use the validation dataset to run predictions against the model and use the results as input to the baseline generation job.

#### The following sample code shows this process for a regression problem. Here, the baseline dataset is generated by running predictions against the model using the validation dataset. This baseline dataset has three different columns – probability, prediction, and label. While probability is the values returned by the model, prediction is inferred from the probability based on a threshold value. label represents the ground truth label from the validation set:

In [ ]:
with open(f"test_data/{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # Header of the file
    for tl in t_lines[1:300]:
        #Remove the first column since it is the label
        test_list = tl.split(",")
        label = test_list.pop(0)
        test_string = ','.join([str(elem) for elem in test_list])
        result = smrt.invoke_endpoint(EndpointName=endpoint_name,
        ContentType="text/csv", Body=test_string)   
        rbody = StreamingBody(raw_stream=result['Body'],content_length=int(result['ResponseMetadata']['HTTPHeaders']['content-length']))
        prediction = rbody.read().decode('utf-8')
        baseline_file.write(f"{prediction},{prediction},{label}\n")
        #print(f"label {label} ; prediction {prediction} ")
        print(".", end="", flush=True)
        sleep(0.5)

#### For model quality monitoring, you use ModelQualityMonitor to configure the infrastructure to execute the processing jobs and the maximum runtime, as shown in the following code:

In [ ]:
# Create the model quality monitoring object
model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)

#### Use the suggest_baseline method to configure and kick off the baseline job. To configure the baseline job, specify where the baseline data is and where you want the baseline results to be saved in S3, as follows:

In [ ]:
# You will specify problem type, in this case Binary Classification, and provide other requirtributes.

job = model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="Regression",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)

job.wait(logs=False)

#### Schedule and execute the model quality monitoring job: The third step is to schedule the model quality monitoring job. To monitor model quality, predictions of the model are first merged with the ground truth inference labels and then compared to the baseline to detect degraded accuracy. Predictions made by the model are already in S3 since data capture is enabled on the endpoint. But how about the ground truth inference labels?

#### The ground truth inference labels would depend on what the model is predicting and what the business use case is. For example, let's say you have a movie recommendation model that you are monitoring. A possible ground truth inference label in this case is whether the user actually watched the recommended movie or not. Maybe the user just clicked on the video but didn't watch it. So, your model-consuming application should have logic to create the ground truth inference labels and upload to an S3 bucket periodically.

#### With the predictions captured and the ground truth inferences provided by your model-consuming application, SageMaker executes a merge job, which is again a periodic job. While scheduling the merge job, take into consideration that the ground truth labels are only available after a certain delay. Once you have the merged data, it's time to monitor the model quality.

#### Here, you create a model quality monitoring job, a job that is executed periodically by SageMaker at a schedule you specify. The code is similar to the scheduling of the data monitoring job, so it is not repeated here. The monitoring job generates statistics and violations and emits CloudWatch metrics. The metrics generated are based on the type of the ML model.

## Bias drift monitoring

#### The concept of bias relates to the individual features of a dataset. Bias is typically measured for sensitive features called facets to identify whether any particular feature or a set of feature values are disproportionately represented in the dataset. Amazon Clarify provides capabilities to detect and monitor bias in a pre-training dataset and deployed models. 

#### 1. Enable data capture for the deployed endpoint: The first step for bias drift monitoring remains the same as other types of monitoring – enabling data capture while deploying a SageMaker endpoint.

#### 2. Generate baseline: The second step is creating a baseline to measure the bias metrics of the training data. A bias drift baseline job needs multiple inputs – the data to use for baselining, the sensitive features, or facets to check for bias, a model to give predictions, and finally, a threshold value to indicate when a model prediction is biased. Let's look at the various configuration objects that capture these details.  

#### Details of the data, such as the location of the validation dataset in the S3 bucket, the type of the dataset (CSV or JSON), and the headers and label of the data, along with the output location of the baseline job results, are captured using DataConfig. Sample code is as follows:

In [ ]:
model_bias_data_config = DataConfig(
    s3_data_input_path=validation_dataset,
    s3_output_path=model_bias_baselining_job_result_uri,
    label=label_header,
    headers=all_headers,
    dataset_type='CSV'
)

#### Details of sensitive features along with threshold values considered as bias are captured by BiasConfig. In the following code, we are monitoring for bias drift in the "City" feature:

In [ ]:
model_bias_config = BiasConfig(
    label_values_or_threshold=[1],
    facet_name="City",
    facet_values_or_threshold=[100],
)

#### To calculate the bias metrics, a deployed model to execute inferences is necessary. ModelConfig captures this model's related information as follows:

In [ ]:
model_config = ModelConfig(
    model_name=model_name,
    instance_count=endpoint_instance_count,
    instance_type=endpoint_instance_type,
    content_type=dataset_type,
    accept_type=dataset_type,
)

#### Finally, ModelPredictedLabelConfig indicates how to extract a predicted label from the model output. For example, the following sample code indicates a prediction of 1 if the probability returned by the model is above 0.8:

In [ ]:
model_predicted_label_config = ModelPredictedLabelConfig(
    probability_threshold=0.8,
)

#### With DataConfig, BiasConfig, ModelConfig, and ModelPredictedLabelConfig in hand, you are ready to create and kick off a baseline job. Sample code is as follows:

In [ ]:
model_bias_monitor = ModelBiasMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

model_bias_monitor.suggest_baseline(
    model_config=model_config,
    data_config=model_bias_data_config,
    bias_config=model_bias_config,
    model_predicted_label_config=model_predicted_label_config,
)

#### During the baseline job execution, SageMaker creates a temporary endpoint called a shadow endpoint. A baselining job runs predictions on the validation dataset, calculates bias metrics, and suggests constraints on these metrics. Once the bias metrics are computed, the shadow endpoint is deleted.

#### Baseline job execution results in a constraints file that shows the bias metric values computed along with the suggested thresholds. 

#### Schedule and execute a model quality monitoring job: The next step is to schedule a bias drift monitoring job. In this step, the monitored bias of the model will be compared against the baseline generated in the previous step. SageMaker executes the bias drift monitoring job using SageMaker Processing periodically according to the schedule you specify. The bias drift monitoring job generates a monitoring report and constraint violations along with CloudWatch metrics.

#### Analyze and act on results: Finally, analyzing the monitoring results and taking remedial actions is similar to the previous monitoring types.

## Best practices for monitoring ML models

- Choosing the correct data format: Model Monitor and Clarify can only monitor for drift in tabular data. Therefore, ensure that your training data is in tabular format. For other data formats, you will have to build custom monitoring containers.
- Choosing real-time endpoints as the mode of model deployment: Model Monitor and Clarify support monitoring for a single-model real-time endpoint. Monitoring a model used with batch transform or multi-model endpoints is not supported. So, ensure that the model you want to monitor is deployed as a single-model real-time endpoint. Additionally, if the model is part of an inference pipeline, the entire pipeline is monitored, not the individual models that make up the pipeline.
- Choosing sampling data capture – sampling percentage: When you enable data capture on a real-time endpoint, a configuration parameter to pay attention to is sampling percentage, which indicates what percentage of the live traffic is captured. Choosing the values for this metric depends on your use case. It is a trade-off between the amount of inference traffic saved and the effectiveness of the model monitoring. If the value of this parameter is close to 100, you have more information stored, leading to more storage costs, and more data for the monitoring job to analyze, leading to a long execution time. On the other hand, a higher sampling percentage leads to capturing more inference traffic patterns to compare against the baseline.
- If your production model is operating in dynamic environments such as retail or financial services, where the consumer behavior or environment factors often change, impacting the model predictions, the best practice is to use a sampling percentage of 100.
- Choosing a dataset for baseline generation: For generating the baseline, the training dataset is typically a good dataset to use. For baseline generation, keep in mind that the first column in the training dataset is considered to be the label. Besides the label, ensure that the number and order of the features in the inference traffic match the training dataset.
- Additionally, for bias drift and feature attribution drift, the baseline generation process stands up a shadow endpoint to collect predictions from. So, consider the limit of the number of active endpoints in your AWS account when executing a baseline job.
- Choosing the monitoring schedule execution frequency: Monitoring jobs, as you have seen so far, are executed on a periodic basis where the minimum interval length is 1 hour. This minimum interval is necessary because enough inference traffic needs to be collected to be compared against the baseline. When determining the monitoring execution frequency, you should select this interval based on the inference traffic your model is serving. For example, a model deployed as part of a busy e-commerce website may serve higher traffic volumes, so running a monitoring job every few hours will give you the chance to detect data and model quality issues quickly. However, every time a monitoring job is executed, it adds to your model monitoring costs. The monitoring job schedule should therefore consider the trade-off between the ability to robustly detect model issues and monitoring costs.
- Scheduling merge and monitoring jobs for model quality monitoring: Model quality monitoring is unique among the four types of monitoring we have discussed in this chapter, in that the model-consuming application should provide ground truth inference labels to be used as part of monitoring. Due to this, you have to consider an additional fact that the model-consuming application may upload the ground truth inference labels using its own schedule. Without the ground truth inference labels in the S3 bucket, the merge job will fail.
- To address this issue, use the StartOffset and EndOffset fields of the ModelQualityJobInput parameter. StartOffset specifies the time subtracted from the start time and EndOffset specifies the time subtracted from the end time of the monitoring job. Offsets are in the format of -P#D, -P#M, or -P#H, where D, M, and H represent days, minutes, and hours, respectively, and # is the number. For example, a -P7H value of StartOffset will cause the monitoring job to start 7 hours after the scheduled time.
- Additionally, ensure that the monitoring schedule cadence is such that any given execution should be completed before the subsequent execution starts, allowing both the ground truth merge job and the monitoring job to complete for each interval.
- Automating remediation actions: While a monitoring solution proactively detects the data and model issues, without a proper plan to act on the issues, you cannot ensure the model's continued ability to meet your business needs. To reap the benefits of the model monitoring alerts generated, as much as possible, automate actions that you need to perform as a result. For example, automate notifications sent to operations and data science teams about possible data and model issues. Similarly, automate collecting or importing new training data and triggering re-training and testing of the models in non-production environments such as dev/QA and staging.
- 